In [2]:
#Data Manipulation Librairies
import pandas as pd
from pandas.plotting import scatter_matrix, andrews_curves
from sklearn.model_selection import train_test_split

#Calculation Utility Librairies
import numpy as np
import scipy.stats as st
import scipy


In [6]:
df_data = pd.read_csv("https://raw.githubusercontent.com/AGuyNextDoor/AProfNextDoor/main/Exercises/Hands%20on%20Machine%20Learning/house-prices-data/dataset.csv")

dataset = df_data.copy()

## Separating Data

In [7]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

dataset_X = dataset.drop(["Id","SalePrice"], axis = 1)
dataset_id = dataset["Id"]
dataset_y = dataset["SalePrice"]

train_dataset_X = train_dataset.drop(["Id","SalePrice"], axis = 1)
train_dataset_id = train_dataset["Id"]
train_dataset_y = train_dataset["SalePrice"]

test_dataset_X = test_dataset.drop(["Id","SalePrice"], axis = 1)
test_dataset_id = test_dataset["Id"]
test_dataset_y = test_dataset["SalePrice"]

# or use the train_test splitter from scikit learn

# Write the example here with sklearn

In [8]:
print("Training dataset has", len(train_dataset_X), "values.")
print("Testing dataset has", len(test_dataset_X), "values.")

Training dataset has 1168 values.
Testing dataset has 292 values.


## Normalization

La normalisation est une transformation de la donnée qui permet de "normaliser" la moyenne et la deviation standard d'une distribution. Pour cela on déplace chaque valeur par l'écart et divisant par la déviation standard. Cela permet de ne pas avoir de valeurs trop de difference entre les differentes features de notre dataset et permet aussi de rendre notre convergence plus stable pour nos modèles. 

In [9]:
train_dataset.describe().transpose()[['mean', 'std']]

,mean,std
Id,730.949486,421.221360
MSSubClass,57.572774,42.053072
MSZoning,3.032534,0.631075
LotFrontage,200.055651,3121.409754
LotArea,10549.443493,9906.540386
...,...,...
MoSold,6.366438,2.701544
YrSold,2007.819349,1.318539
SaleType,7.482021,1.585502
SaleCondition,3.761130,1.134621


### Normalization with TensorFlow 

Normalization can be added directy in our models so that we don't have to worry about it anymore !

In [10]:
from tensorflow.keras.layers.experimental import preprocessing

normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_dataset_X))
print(normalizer.mean.numpy())

[5.75727577e+01 3.03253508e+00 2.00055664e+02 1.05494443e+04
 9.96575475e-01 1.94606161e+00 2.76626730e+00 8.56164377e-04
 3.06678057e+00 6.07876703e-02 1.23202047e+01 2.03253388e+00
 2.01027393e+00 4.88013774e-01 3.04794431e+00 6.15325212e+00
 5.58647346e+00 1.97117981e+03 1.98508813e+03 1.40924621e+00
 1.07619846e+00 9.56164265e+00 1.03047962e+01 1.76626718e+00
 1.05862999e+02 2.52654076e+00 3.73544526e+00 1.39041102e+00
 2.27911019e+00 2.83304763e+00 2.29623318e+00 2.82191706e+00
 4.43898987e+02 4.74914455e+00 4.65042763e+01 5.75950378e+02
 1.06635352e+03 1.03681505e+00 1.51369882e+00 9.34075296e-01
 3.69948649e+00 1.16843384e+03 3.55280823e+02 6.92465782e+00
 1.53063940e+03 4.29794461e-01 5.65068498e-02 1.56763673e+00
 3.91267031e-01 2.86130071e+00 1.04708874e+00 2.32705450e+00
 6.55308199e+00 5.76027489e+00 6.21575415e-01 3.77910924e+00
 2.51712322e+00 1.88355994e+03 1.27825356e+00 1.78424644e+00
 4.77242340e+02 3.93321848e+00 3.96575356e+00 1.86301351e+00
 9.43647156e+01 4.702567

## Deep Neural Networks

In [11]:
import tensorflow as tf
import datetime, os

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation, Multiply
from keras.callbacks import TensorBoard

from tensorboard import notebook
from tensorboard.plugins.hparams import api as hp

%reload_ext tensorboard

#HParam
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([64, 128, 256]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.05, 0.1))

HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['nadam', 'adamax']))

METRIC_ACCURACY = 'Accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

### Creating custom hyperparameter functions

In [12]:
def lr_schedule(epoch):
  """
  Returns a custom learning rate that decreases as epochs progress.
  """
  learning_rate = 0.2
  if epoch > (total_epochs*0.05):
    learning_rate = 0.1
  if epoch > (total_epochs*0.4):
    learning_rate = 0.02
  if epoch > (total_epochs*0.6):
    learning_rate = 0.01
    
  tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
  return learning_rate


In [13]:
def create_model(hparams, tot_epochs, tensorboard_callback, run_name, X, y, Xtest, ytest, inputDim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS],
                              input_dim=inputDim,
                              activation='relu',
                              kernel_initializer='random_uniform',
                              bias_initializer='zeros'),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(128,
                              activation="relu",
                              kernel_initializer='random_uniform',
                              bias_initializer='zeros'),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(128,
                              activation="relu",
                              kernel_initializer='random_uniform',
                              bias_initializer='zeros'),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(128,
                              activation="relu",
                              kernel_initializer='random_uniform',
                              bias_initializer='zeros'),
        tf.keras.layers.Dense(64,
                              activation="relu",
                              kernel_initializer='random_uniform',
                              bias_initializer='zeros'),
        tf.keras.layers.Dense(1, activation='linear')
    ])

    model.compile(loss="mse",
                  optimizer=hparams[HP_OPTIMIZER],
                  metrics=['mae', 'mape', 'accuracy'])


    # Fit Model
    model.fit(
        X,
        y,
        epochs=tot_epochs,
        batch_size=32,
        validation_data=(Xtest, ytest),
        shuffle=False,
        callbacks=[tensorboard_callback,
                   hp.KerasCallback(run_name, hparams)])

    #accuracy = mean_percentage(y_test_energy, model.predict(X_test))
    mse, mape = evaluate_tf(model, Xtest, ytest)
    #pred = model.predict(X_test).tolist()
    #Pred = []
    #for sublist in pred:
    #    for item in sublist:
    #        Pred.append(item)
    
    #mape = mean_percentage_np(y_test_energy.tolist(), Pred)
    #mse = np.sqrt(K.get_value(K.mean(K.square(np.subtract(Pred, y_test_energy.tolist())))))
    #perc = K.mean(100*(Divide()([K.abs(np.subtract(y_test_energy.tolist(), Pred)), y_test_energy.tolist()])))
    
    print("mse here is : ", mse)
    m_dict = {
        'model': model,
        'test_mse': mse,
        'test_mape' : mape,
        'description': {h.name: hparams[h] for h in hparams}
    }
    all_models.append(m_dict)
    return mse


In [14]:
def evaluate_tf(model, test_features, test_labels):
    predictions_list = model.predict(test_features)
    predictions = []
    for sublist in predictions_list:
        for item in sublist:
            predictions.append(item)

    errors = abs(predictions - test_labels)

    mape = 100 * np.mean(np.divide(errors, test_labels))

    mse = np.sqrt(np.mean(np.square(np.subtract(predictions, test_labels))))
    return mse, mape

In [15]:
tf.keras.backend.clear_session()

!rm -rf ./logs/
!rm -rf ./fit/

all_models = []
session_num = 0
total_epochs = 100
run_name = "logs/fit/" + ("run-%d" % session_num)


for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "logs/fit/" + ("run-%d" % session_num)

      tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=run_name,
                                                      histogram_freq=1)
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      descript = {h.name: hparams[h] for h in hparams}
      #run('logs/hparam_tuning/' + run_name, hparams)
      create_model(hparams, total_epochs,tensorboard_callback, run_name, train_dataset_X, train_dataset_y , test_dataset_X, test_dataset_y, inputDim = len(train_dataset_X.iloc[0]))
      session_num += 1
all_models_energy = all_models

--- Starting trial: logs/fit/run-0
{'num_units': 64, 'dropout': 0.05, 'optimizer': 'adamax'}
Epoch 1/100
37/37 [==============================] - 1s 7ms/step - loss: 39878672384.0000 - mae: 182478.0625 - mape: 99.4200 - accuracy: 0.0000e+00 - val_loss: 31893129216.0000 - val_mae: 164426.1406 - val_mape: 95.9629 - val_accuracy: 0.0000e+00
Epoch 2/100
37/37 [==============================] - 0s 2ms/step - loss: 22731173888.0000 - mae: 123285.2344 - mape: 64.8138 - accuracy: 0.0000e+00 - val_loss: 15257330688.0000 - val_mae: 50975.4922 - val_mape: 30.0914 - val_accuracy: 0.0000e+00
Epoch 3/100
37/37 [==============================] - 0s 2ms/step - loss: 7325169664.0000 - mae: 49803.8594 - mape: 28.3330 - accuracy: 0.0000e+00 - val_loss: 15341135872.0000 - val_mae: 49498.5117 - val_mape: 29.8619 - val_accuracy: 0.0000e+00
Epoch 4/100
37/37 [==============================] - 0s 2ms/step - loss: 5917876736.0000 - mae: 47176.8984 - mape: 26.2181 - accuracy: 0.0000e+00 - val_loss: 16764113920.

Epoch 68/100
37/37 [==============================] - 0s 2ms/step - loss: 1917259264.0000 - mae: 28294.6543 - mape: 16.7410 - accuracy: 0.0000e+00 - val_loss: 2602658560.0000 - val_mae: 26826.1426 - val_mape: 17.0012 - val_accuracy: 0.0000e+00
Epoch 69/100
37/37 [==============================] - 0s 2ms/step - loss: 2006341376.0000 - mae: 28745.2598 - mape: 16.8176 - accuracy: 0.0000e+00 - val_loss: 2616460544.0000 - val_mae: 27219.0488 - val_mape: 17.3063 - val_accuracy: 0.0000e+00
Epoch 70/100
37/37 [==============================] - 0s 1ms/step - loss: 1832432640.0000 - mae: 28154.4688 - mape: 16.5945 - accuracy: 0.0000e+00 - val_loss: 2450564096.0000 - val_mae: 26901.7578 - val_mape: 17.0426 - val_accuracy: 0.0000e+00
Epoch 71/100
37/37 [==============================] - 0s 2ms/step - loss: 2062390144.0000 - mae: 28771.1367 - mape: 16.8491 - accuracy: 0.0000e+00 - val_loss: 2535619328.0000 - val_mae: 26709.0879 - val_mape: 16.9546 - val_accuracy: 0.0000e+00
Epoch 72/100
37/37 [====

Epoch 1/100
37/37 [==============================] - 1s 10ms/step - loss: 25512855552.0000 - mae: 120781.7422 - mape: 65.5643 - accuracy: 0.0000e+00 - val_loss: 16838848512.0000 - val_mae: 50838.4297 - val_mape: 30.3499 - val_accuracy: 0.0000e+00
Epoch 2/100
37/37 [==============================] - 0s 2ms/step - loss: 6171128832.0000 - mae: 47277.0039 - mape: 25.9940 - accuracy: 0.0000e+00 - val_loss: 18581317632.0000 - val_mae: 50434.6641 - val_mape: 33.8515 - val_accuracy: 0.0000e+00
Epoch 3/100
37/37 [==============================] - 0s 2ms/step - loss: 4862644736.0000 - mae: 44457.6758 - mape: 25.6808 - accuracy: 0.0000e+00 - val_loss: 18964375552.0000 - val_mae: 49060.6406 - val_mape: 33.9123 - val_accuracy: 0.0000e+00
Epoch 4/100
37/37 [==============================] - 0s 2ms/step - loss: 3998783232.0000 - mae: 42944.2617 - mape: 25.5829 - accuracy: 0.0000e+00 - val_loss: 17807527936.0000 - val_mae: 46493.0977 - val_mape: 32.4894 - val_accuracy: 0.0000e+00
Epoch 5/100
37/37 [==

Epoch 68/100
37/37 [==============================] - 0s 2ms/step - loss: 1445398144.0000 - mae: 26751.7852 - mape: 15.9942 - accuracy: 0.0000e+00 - val_loss: 4275080704.0000 - val_mae: 26452.9238 - val_mape: 17.3453 - val_accuracy: 0.0000e+00
Epoch 69/100
37/37 [==============================] - 0s 2ms/step - loss: 1514423680.0000 - mae: 26957.5078 - mape: 15.9667 - accuracy: 0.0000e+00 - val_loss: 4580743168.0000 - val_mae: 26514.0293 - val_mape: 17.4813 - val_accuracy: 0.0000e+00
Epoch 70/100
37/37 [==============================] - 0s 2ms/step - loss: 1493755776.0000 - mae: 26822.2695 - mape: 15.8552 - accuracy: 0.0000e+00 - val_loss: 4584456704.0000 - val_mae: 28700.6855 - val_mape: 19.0732 - val_accuracy: 0.0000e+00
Epoch 71/100
37/37 [==============================] - 0s 2ms/step - loss: 1427659904.0000 - mae: 26405.8965 - mape: 15.7719 - accuracy: 0.0000e+00 - val_loss: 4245748736.0000 - val_mae: 25873.1230 - val_mape: 17.0060 - val_accuracy: 0.0000e+00
Epoch 72/100
37/37 [====

37/37 [==============================] - 0s 2ms/step - loss: 2133056128.0000 - mae: 30830.1445 - mape: 17.9521 - accuracy: 0.0000e+00 - val_loss: 4348067840.0000 - val_mae: 29741.2812 - val_mape: 19.1169 - val_accuracy: 0.0000e+00
Epoch 35/100
37/37 [==============================] - 0s 2ms/step - loss: 2455343872.0000 - mae: 31324.2637 - mape: 18.1644 - accuracy: 0.0000e+00 - val_loss: 4373553664.0000 - val_mae: 30370.4648 - val_mape: 19.6142 - val_accuracy: 0.0000e+00
Epoch 36/100
37/37 [==============================] - 0s 2ms/step - loss: 2179857920.0000 - mae: 30553.9902 - mape: 17.7130 - accuracy: 0.0000e+00 - val_loss: 4062599424.0000 - val_mae: 29168.4863 - val_mape: 18.6576 - val_accuracy: 0.0000e+00
Epoch 37/100
37/37 [==============================] - 0s 2ms/step - loss: 2335241472.0000 - mae: 31898.1445 - mape: 18.3936 - accuracy: 0.0000e+00 - val_loss: 4075628288.0000 - val_mae: 29304.5957 - val_mape: 18.7602 - val_accuracy: 0.0000e+00
Epoch 38/100
37/37 [=================

...

Epoch 68/100
37/37 [==============================] - 0s 3ms/step - loss: 1881578240.0000 - mae: 28365.2031 - mape: 16.7293 - accuracy: 0.0000e+00 - val_loss: 2432791040.0000 - val_mae: 27929.9277 - val_mape: 17.9426 - val_accuracy: 0.0000e+00
Epoch 69/100
37/37 [==============================] - 0s 3ms/step - loss: 1884462720.0000 - mae: 28366.4785 - mape: 16.8301 - accuracy: 0.0000e+00 - val_loss: 2245966848.0000 - val_mae: 26886.6309 - val_mape: 17.1874 - val_accuracy: 0.0000e+00
Epoch 70/100
37/37 [==============================] - 0s 3ms/step - loss: 1857093760.0000 - mae: 28309.6152 - mape: 16.8393 - accuracy: 0.0000e+00 - val_loss: 2072766336.0000 - val_mae: 26428.0918 - val_mape: 16.7876 - val_accuracy: 0.0000e+00
Epoch 71/100
37/37 [==============================] - 0s 3ms/step - loss: 1839941888.0000 - mae: 28266.8633 - mape: 16.6531 - accuracy: 0.0000e+00 - val_loss: 2120258176.0000 - val_mae: 27125.9844 - val_mape: 17.2922 - val_accuracy: 0.0000e+00
Epoch 72/100
37/37 [====

Epoch 35/100
37/37 [==============================] - 0s 2ms/step - loss: 1821968512.0000 - mae: 28939.7188 - mape: 17.0738 - accuracy: 0.0000e+00 - val_loss: 2266268672.0000 - val_mae: 26367.8125 - val_mape: 16.9952 - val_accuracy: 0.0000e+00
Epoch 36/100
37/37 [==============================] - 0s 2ms/step - loss: 1786019456.0000 - mae: 28627.6523 - mape: 16.9198 - accuracy: 0.0000e+00 - val_loss: 2407259648.0000 - val_mae: 26913.1934 - val_mape: 17.3395 - val_accuracy: 0.0000e+00
Epoch 37/100
37/37 [==============================] - 0s 2ms/step - loss: 1828850816.0000 - mae: 28486.3145 - mape: 16.7695 - accuracy: 0.0000e+00 - val_loss: 2261279744.0000 - val_mae: 26147.8320 - val_mape: 16.8480 - val_accuracy: 0.0000e+00
Epoch 38/100
37/37 [==============================] - 0s 2ms/step - loss: 1688193792.0000 - mae: 27689.2754 - mape: 16.4723 - accuracy: 0.0000e+00 - val_loss: 2322354688.0000 - val_mae: 27098.1602 - val_mape: 17.4933 - val_accuracy: 0.0000e+00
Epoch 39/100
37/37 [====

37/37 [==============================] - 1s 9ms/step - loss: 38324518912.0000 - mae: 178462.6250 - mape: 97.0790 - accuracy: 0.0000e+00 - val_loss: 24076058624.0000 - val_mae: 139808.8125 - val_mape: 80.2109 - val_accuracy: 0.0000e+00
Epoch 2/100
37/37 [==============================] - 0s 2ms/step - loss: 13172405248.0000 - mae: 72682.6562 - mape: 39.0050 - accuracy: 0.0000e+00 - val_loss: 16797474816.0000 - val_mae: 53208.0430 - val_mape: 32.7141 - val_accuracy: 0.0000e+00
Epoch 3/100
37/37 [==============================] - 0s 2ms/step - loss: 7738996224.0000 - mae: 49971.9492 - mape: 26.7440 - accuracy: 0.0000e+00 - val_loss: 15350089728.0000 - val_mae: 49779.7539 - val_mape: 30.1990 - val_accuracy: 0.0000e+00
Epoch 4/100
37/37 [==============================] - 0s 2ms/step - loss: 6466687488.0000 - mae: 47599.7930 - mape: 26.3400 - accuracy: 0.0000e+00 - val_loss: 16004196352.0000 - val_mae: 48939.0977 - val_mape: 30.7525 - val_accuracy: 0.0000e+00
Epoch 5/100
37/37 [=============

Epoch 68/100
37/37 [==============================] - 0s 2ms/step - loss: 1900646528.0000 - mae: 29115.2637 - mape: 17.0739 - accuracy: 0.0000e+00 - val_loss: 3297929728.0000 - val_mae: 27514.7578 - val_mape: 17.5147 - val_accuracy: 0.0000e+00
Epoch 69/100
37/37 [==============================] - 0s 2ms/step - loss: 1817614976.0000 - mae: 28627.5352 - mape: 16.9551 - accuracy: 0.0000e+00 - val_loss: 3125545728.0000 - val_mae: 27209.9062 - val_mape: 17.2661 - val_accuracy: 0.0000e+00
Epoch 70/100
37/37 [==============================] - 0s 2ms/step - loss: 2024798464.0000 - mae: 28908.0039 - mape: 16.8398 - accuracy: 0.0000e+00 - val_loss: 3416862208.0000 - val_mae: 27975.9629 - val_mape: 17.9004 - val_accuracy: 0.0000e+00
Epoch 71/100
37/37 [==============================] - 0s 2ms/step - loss: 1900448000.0000 - mae: 29028.9043 - mape: 17.1337 - accuracy: 0.0000e+00 - val_loss: 3133626624.0000 - val_mae: 26941.4336 - val_mape: 17.0935 - val_accuracy: 0.0000e+00
Epoch 72/100
37/37 [====

Epoch 35/100
37/37 [==============================] - 0s 2ms/step - loss: 1792112512.0000 - mae: 29590.4961 - mape: 17.4839 - accuracy: 0.0000e+00 - val_loss: 2823125248.0000 - val_mae: 30372.2285 - val_mape: 19.6947 - val_accuracy: 0.0000e+00
Epoch 36/100
37/37 [==============================] - 0s 2ms/step - loss: 1829359360.0000 - mae: 29781.5078 - mape: 17.5968 - accuracy: 0.0000e+00 - val_loss: 2757457408.0000 - val_mae: 30187.3633 - val_mape: 19.5899 - val_accuracy: 0.0000e+00
Epoch 37/100
37/37 [==============================] - 0s 2ms/step - loss: 1788707456.0000 - mae: 28665.8730 - mape: 17.0713 - accuracy: 0.0000e+00 - val_loss: 2475333376.0000 - val_mae: 26469.0938 - val_mape: 16.9529 - val_accuracy: 0.0000e+00
Epoch 38/100
37/37 [==============================] - 0s 2ms/step - loss: 1820183552.0000 - mae: 28953.8672 - mape: 17.1120 - accuracy: 0.0000e+00 - val_loss: 2268497920.0000 - val_mae: 24656.6680 - val_mape: 15.5442 - val_accuracy: 0.0000e+00
Epoch 39/100
37/37 [====

In [16]:
%tensorboard --logdir logs